# Query table colume relationships 


## Install dependencies
```shell
pip install graphiti-core langchain-openai langgraph ipywidgets
```

Ensure that you've followed the Graphiti installation instructions. In particular, installation of `neo4j`.

In [1]:
%pip install graphiti-core langchain-openai langgraph ipywidgets

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


## Run agent

In [2]:
from logger import logger

from rag_graphiti_client import client,llm

from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import END, START, StateGraph
from langgraph.prebuilt import ToolNode
from rag_graphiti_mcp_tools import get_table_columns

tools = [get_table_columns]
tool_node = ToolNode(tools)

llm_with_tools = llm.bind_tools(tools)

from rag_graphiti_chatbot import Chatbot,State
chatbot = Chatbot(
    llm=llm_with_tools,
    graphiti=client
)

graph_builder = StateGraph(State)

memory = MemorySaver()

# Define the function that determines whether to continue or not
async def should_continue(state, config):
    # logger.info(f"should_continue state: {state}")
    messages = state['messages']
    last_message = messages[-1]
    # If there is no function call, then we finish
    if not last_message.tool_calls:
        return 'end'
    # Otherwise if there is, we continue
    else:
        return 'continue'

graph_builder.add_node('agent', chatbot.generate_chatbot_response)
graph_builder.add_node('tools', tool_node)

graph_builder.add_edge(START, 'agent')
graph_builder.add_conditional_edges('agent', should_continue, {'continue': 'tools', 'end': END})
graph_builder.add_edge('tools', 'agent')

graph = graph_builder.compile(checkpointer=memory)

from rag_graphiti_user import query_user,create_user
user_name = "tcq"
user_node_uuid = await query_user(user_name)
if user_node_uuid is None:
    user_node_uuid = await create_user(user_name)
user_state = State(
    user_name=user_name, 
    user_node_uuid=user_node_uuid
)

from rag_graphiti_agent import Agent,AgentRun
agent = Agent(graph)
agentRun:AgentRun = agent.run(user_state=user_state)

import ipywidgets as widgets
from IPython.display import display
conversation_output = widgets.Output()
input_box = widgets.Text(placeholder='Type your message here...')
submit_button = widgets.Button(description='Send')

import asyncio
def on_submit(b):
    user_input = input_box.value
    input_box.value = ''
    conversation_output.append_stdout(f'\nUser: {user_input}\n')
    conversation_output.append_stdout('\nAssistant: ')
    asyncio.create_task(agentRun.process_user_message(user_input)) 
submit_button.on_click(on_submit)
display(widgets.VBox([input_box, submit_button, conversation_output]))

def update_conversation_output():
    while True:
        system_message = agentRun.system_message_output.get_stdout()
        if system_message != None:
            conversation_output.append_stdout(system_message)
import threading
threading.Thread(target=update_conversation_output).start()
